# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hauterive,46.0901,3.4459,15.42,85,67,3.06,FR,1727395099
1,1,edinburgh of the seven seas,-37.0676,-12.3116,13.67,93,98,11.65,SH,1727395100
2,2,invercargill,-46.4000,168.3500,3.90,93,92,1.34,NZ,1727395101
3,3,whitehorse,60.7161,-135.0538,9.50,49,40,2.57,CA,1727395102
4,4,bethel,41.3712,-73.4140,21.93,89,100,0.45,US,1727395104


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    legend='right',
    c='City', 
    cmap='Category10'
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [69]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temperature_range = [21.0, 27.0]
ideal_wind_max = 10
cloudness_max = 20.0
city_data_df_ideal = city_data_df[(city_data_df['Max Temp'] >= ideal_temperature_range[0]) & (city_data_df['Max Temp'] <= ideal_temperature_range[1])]
city_data_df_ideal = city_data_df_ideal[city_data_df_ideal['Wind Speed'] <= ideal_wind_max]
city_data_df_ideal = city_data_df_ideal[city_data_df_ideal['Cloudiness'] <= cloudness_max]

# Drop any rows with null values
city_data_df_cleaned = city_data_df_ideal.dropna()

# Display sample data
city_data_df_cleaned.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,mount isa,-20.7333,139.5000,21.87,46,3,6.17,AU,1727395108
16,16,mitchell,43.7094,-98.0298,26.76,41,1,3.58,US,1727395118
42,42,uturoa,-16.7333,-151.4333,26.80,79,6,4.36,PF,1727395151
49,49,lompoc,34.6391,-120.4579,21.45,70,5,3.13,US,1727395159
64,64,west island,-12.1568,96.8225,25.99,83,20,5.66,CC,1727395179


### Step 3: Create a new DataFrame called `hotel_df`.

In [70]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_cleaned[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = hotel_df.apply(lambda _: ' ', axis=1)

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,mount isa,AU,-20.7333,139.5000,46,
16,mitchell,US,43.7094,-98.0298,41,
42,uturoa,PF,-16.7333,-151.4333,79,
49,lompoc,US,34.6391,-120.4579,70,
64,west island,CC,-12.1568,96.8225,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [71]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    print(latitude)

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
-20.7333
mount isa - nearest hotel: Ibis Styles
43.7094
mitchell - nearest hotel: Quality Inn
-16.7333
uturoa - nearest hotel: Hawaiki Nui hotel
34.6391
lompoc - nearest hotel: Red Roof Inn Lompoc
-12.1568
west island - nearest hotel: Cocos Village Bungalows
5.4112
georgetown - nearest hotel: Page 63 hostel
-23.865
inhambane - nearest hotel: Hotel de Inhambane
42.3505
twin falls - nearest hotel: No hotel found
38.1811
argostolion - nearest hotel: Ainos Hotel
49.1334
estevan - nearest hotel: Days Inn
-22.9456
thohoyandou - nearest hotel: Khoroni Hotel Casino and Convention Resort
28.629
tazacorte - nearest hotel: App Leyma
-17.8919
nova vicosa - nearest hotel: Pousada Pontal da Barra
-18.65
neiafu - nearest hotel: Port Wine Guest House
-19.6833
port mathurin - nearest hotel: Escale Vacances
36.6777
salinas - nearest hotel: The Traveler's Hotel
-0.6136
salinopolis - nearest hotel: Hotel Salinópolis
37.8398
fort scott - nearest hotel: The Courtland Hotel & Spa
45.416

,City,Country,Lat,Lng,Humidity,Hotel Name
8,mount isa,AU,-20.7333,139.5000,46,Ibis Styles
16,mitchell,US,43.7094,-98.0298,41,Quality Inn
42,uturoa,PF,-16.7333,-151.4333,79,Hawaiki Nui hotel
49,lompoc,US,34.6391,-120.4579,70,Red Roof Inn Lompoc
64,west island,CC,-12.1568,96.8225,83,Cocos Village Bungalows
73,georgetown,MY,5.4112,100.3354,87,Page 63 hostel
74,inhambane,MZ,-23.8650,35.3833,78,Hotel de Inhambane
76,twin falls,US,42.3505,-114.6445,21,No hotel found
105,argostolion,GR,38.1811,20.4890,48,Ainos Hotel
107,estevan,CA,49.1334,-102.9842,18,Days Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [72]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    legend='right',
    c='City', 
    cmap='Category10',
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)